# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterMessage", engine)
X = df['message'].values
Y = df[df.columns[4:]].values

### 2. Write a tokenization function to process your text data

In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state=0))))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
import nltk
nltk.download(['punkt', 'wordnet'])

X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...nalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)
classification_report(np.hstack(y_test), np.hstack(y_pred), target_names=list(df.columns[4:]))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))


'                        precision    recall  f1-score   support\n\n               related       0.96      0.98      0.97    215354\n               request       0.77      0.61      0.68     20541\n                 offer       0.77      0.20      0.32        49\n\n           avg / total       0.95      0.95      0.95    235944\n'

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
                'tfidf__smooth_idf': [True, False],
                'clf__estimator__estimator__C': [1, 5]
             }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [8]:
from sklearn.metrics import accuracy_score

for i, category in enumerate(df.columns[4:]):
    print(category, accuracy_score(y_test[:, i], y_pred[:, i]))

related 0.822551113824
request 0.896856881294
offer 0.995422642661
aid_related 0.763045468416
medical_help 0.928135489777
medical_products 0.959108941105
search_and_rescue 0.975434848947
security 0.983063777846
military 0.971162648764
child_alone 1.0
water 0.961550198352
food 0.945834604821
shelter 0.946750076289
clothing 0.990082392432
money 0.974366798901
missing_people 0.989319499542
refugees 0.971620384498
death 0.968568812939
other_aid 0.862221544095
infrastructure_related 0.93240768996
transport 0.961245041196
buildings 0.961550198352
electricity 0.97970704913
tools 0.994507171193
hospitals 0.98870918523
shops 0.995117485505
aid_centers 0.989166920964
other_infrastructure 0.953310955142
weather_related 0.87686908758
floods 0.959871833995
storm 0.942783033262
fire 0.990692706744
earthquake 0.967958498627
cold 0.984284406469
other_weather 0.952090326518
direct_report 0.869697894416


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
import warnings
warnings.filterwarnings("ignore")
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
classification_report(np.hstack(y_test), np.hstack(y_pred), target_names=list(df.columns[4:]))

'                        precision    recall  f1-score   support\n\n               related       0.96      0.98      0.97    215354\n               request       0.77      0.61      0.68     20541\n                 offer       0.77      0.20      0.32        49\n\n           avg / total       0.95      0.95      0.95    235944\n'

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
print(classification_report(np.hstack(y_test), np.hstack(y_pred), target_names=list(df.columns[4:])))

                        precision    recall  f1-score   support

               related       0.96      0.98      0.97    215354
               request       0.77      0.61      0.68     20541
                 offer       0.77      0.20      0.32        49

           avg / total       0.95      0.95      0.95    235944



In [11]:
for i, category in enumerate(df.columns[4:]):
    print(category, accuracy_score(y_test[:, i], y_pred[:, i]))

related 0.822551113824
request 0.896856881294
offer 0.995422642661
aid_related 0.763045468416
medical_help 0.928135489777
medical_products 0.959108941105
search_and_rescue 0.975434848947
security 0.983063777846
military 0.971162648764
child_alone 1.0
water 0.961550198352
food 0.945834604821
shelter 0.946750076289
clothing 0.990082392432
money 0.974366798901
missing_people 0.989319499542
refugees 0.971620384498
death 0.968568812939
other_aid 0.862221544095
infrastructure_related 0.93240768996
transport 0.961245041196
buildings 0.961550198352
electricity 0.97970704913
tools 0.994507171193
hospitals 0.98870918523
shops 0.995117485505
aid_centers 0.989166920964
other_infrastructure 0.953310955142
weather_related 0.87686908758
floods 0.959871833995
storm 0.942783033262
fire 0.990692706744
earthquake 0.967958498627
cold 0.984284406469
other_weather 0.952090326518
direct_report 0.869697894416


### 9. Export your model as a pickle file

In [12]:
from sklearn.externals import joblib

joblib.dump(cv, 'classifier.pkl')

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.